In [0]:
import os
import pandas as pd
import numpy as np
import SimpleITK as sitk
import pywt
import cv2

def fuseCoeff(cooef1, cooef2,cooef3, cooef4, method):

    if (method == 'mean'):
        cooef = (cooef1 + cooef2+ cooef3+ cooef4) / 4
    elif (method == 'min'):
        cooef = np.minimum(cooef1,cooef2)
        cooef = np.minimum(cooef,cooef3)
        cooef = np.minimum(cooef,cooef4)
    elif (method == 'max'):
        cooef = np.maximum(cooef1, cooef2)
        cooef = np.maximum(cooef, cooef3)
        cooef = np.maximum(cooef, cooef4)
    else:
        cooef = []

    return cooef

FUSION_METHOD = '' # Can be 'min' || 'max || anything you choose according theory

inp1= sitk.ReadImage("")#path to Flair
inp2= sitk.ReadImage("")#Path to T1
inp3 = sitk.ReadImage("")#path to T1c
inp4 = sitk.ReadImage("")#path to T2

# Read the two image
I1 = sitk.GetArrayFromImage(inp1)
I2 = sitk.GetArrayFromImage(inp2)
I3 = sitk.GetArrayFromImage(inp3)
I4 = sitk.GetArrayFromImage(inp4)

# First: Do wavelet transform on each image
wavelet = 'db1'
cooef1 = pywt.wavedec2(I1[:,:], wavelet)
cooef2 = pywt.wavedec2(I2[:,:], wavelet)
cooef3 = pywt.wavedec2(I3[:,:], wavelet)
cooef4 = pywt.wavedec2(I4[:,:], wavelet)

# Second: for each level in both image do the fusion according to the desire option
fusedCooef = []

for i in range(len(cooef1)-1):

    # The first values in each decomposition is the apprximation values of the top level
    if(i == 0):

        fusedCooef.append(fuseCoeff(cooef1[0],cooef2[0], cooef3[0], cooef4[0], FUSION_METHOD))

    else:

        # For the rest of the levels we have tupels with 3 coeeficents
        c1 = fuseCoeff(cooef1[i][0], cooef2[i][0], cooef3[i][0], cooef4[i][0], FUSION_METHOD)
        c2 = fuseCoeff(cooef1[i][1], cooef2[i][1], cooef3[i][1], cooef4[i][1], FUSION_METHOD)
        c3 = fuseCoeff(cooef1[i][2], cooef2[i][2], cooef3[i][2], cooef4[i][2], FUSION_METHOD)

        fusedCooef.append((c1,c2,c3))

# Third: After we fused the cooefficent we nned to transfor back to get the image
fusedImage = pywt.waverec2(fusedCooef, wavelet)

# Forth: normmalize values to be in uint8
fusedImage = np.multiply(np.divide(fusedImage - np.min(fusedImage),(np.max(fusedImage) - np.min(fusedImage))),255)
fusedImage = fusedImage.astype(np.uint8)

#fusedImage=sitk.ReadImage(fusedImage)


fusedImage=sitk.GetImageFromArray(fusedImage)
